<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Llama_3_1_8B_Instruct_space_travel_demo_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 OR L4 IN GOOGLE COLAB
#!pip install -U transformers
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

In [1]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)

In [ ]:
import os
from typing import List, Dict

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

import logging

# Disable warnings from transformers
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

import torch
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")


# Specify CUDA device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize Llama model and tokenizer
llama_model_id = "meta-llama/Llama-3.1-8B-Instruct"

# Initialize Llama 3 with padding token fix
llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_id, use_fast=True)
llama_tokenizer.padding_side = "right"
llama_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Initialize Llama 3
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
llama_model = AutoModelForCausalLM.from_pretrained(
    llama_model_id,
    device_map=device,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

# Resize token embeddings with mean_resizing=False
llama_model.resize_token_embeddings(len(llama_tokenizer), mean_resizing=False)

# Move model to the device
llama_model.to(device)

llama_pipeline = pipeline(
    "text-generation",
    model=llama_model,
    tokenizer=llama_tokenizer,
    pad_token_id=llama_tokenizer.pad_token_id
)

llama_pipeline.generation_params = {
    "max_new_tokens": 4096,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "temperature": 0.7,
    "pad_token_id": llama_tokenizer.pad_token_id,
}

# Define a function to interact with the LLM
def generate_flight_plan_with_llm(prompt: str, system_prompt: str = None, model: str = llama_model_id, temperature: float = 0.7, max_tokens: int = 2048) -> str:
    full_prompt = prompt  # Start with the user's prompt
    if system_prompt:
        full_prompt = f"{system_prompt}\n{prompt}"  # Prepend the system prompt

    # Generate text with Llama
    #response = llama_pipeline(full_prompt, max_length=max_tokens, temperature=temperature, num_return_sequences=1)
    response = llama_pipeline(full_prompt, max_length=max_tokens, temperature=temperature, num_return_sequences=1, truncation=True)

    flight_plan = response[0]['generated_text'].replace(full_prompt, "").strip()  # Extract generated content
    return flight_plan

## EARTH TO MARS

In [3]:
# Define a class for the AI agent
class SpaceFlightPlanningAgent:
    def __init__(self, model=llama_model_id):
        self.model = model

    def generate_flight_plan(
            self,
            origin: str,
            destination: str,
            spacecraft: str,
            mission_objectives: List[str],
            constraints: List[str],
    ) -> Dict[str, Dict[str, str]]:
        system_prompt = f"""
        You are a helpful and informative space flight planner.

        You are tasked with generating a detailed flight plan for a mission from {origin} to {destination}
        using a {spacecraft}. The mission objectives are: {mission_objectives}. The flight plan
        must satisfy the following constraints: {constraints}.

        Consider these factors:
        - Orbital mechanics (including Hohmann transfer orbits, delta-v calculations, and
          apogee/perigee)
        - Spacecraft capabilities (delta-v, thrust-to-weight ratio, specific impulse)
        - Gravitational forces of relevant celestial bodies (Sun, Earth, Mars, etc.)
        - Radiation exposure (minimize exposure by considering solar activity and trajectory)
        - Communication windows (ensure contact with Earth throughout the mission)
        - Launch opportunities (identify optimal launch windows based on planetary alignment)
        """

        flight_plan_sections = {
            "Launch Date and Time": """Provide a **specific** launch date and time in UTC, considering optimal launch windows and radiation exposure.
                                 Do not provide a range of dates or suggest future windows. Choose a date and time within the next 5 years.""",  # Modified prompt
            "Trajectory": {  # Trajectory section with iterative prompting strategy
                "initial_prompt": "Provide a summary of the key phases and maneuvers of the trajectory.",
                "follow_up_prompts": [
                    "Describe the launch process, including the type of launch vehicle, the target orbit, and any relevant maneuvers. Provide details about the initial orbit, such as its altitude, inclination, and purpose.",
                    "Explain the Hohmann transfer maneuver, including the delta-v requirements, the timing of the maneuver, and the resulting trajectory.",
                    "Detail the interplanetary cruise phase, including the duration, the communication strategy, and any planned course corrections.",
                    "Describe the approach to Mars and the Mars orbit insertion maneuver, including the delta-v requirements, the target orbit parameters, and any challenges or considerations."
                ]
            },
            "Maneuver Schedule": "Detail each maneuver with its purpose, delta-v requirement, and timing. Include maneuvers like trans-Mars injection, Mars orbit insertion, and any course correction burns.",
            "Communication Plan": "Explain how communication with Earth will be maintained throughout the mission, including the use of the Deep Space Network and any considerations for communication delays or blackouts.",
            "Contingency Plans": "Describe contingency plans for various scenarios, such as launch vehicle failure, missed maneuver opportunities, or spacecraft malfunctions. Address crew safety and mission recovery options."
        }

        flight_plan = {}  # Store flight plan sections for current model
        for section_name, section_instructions in flight_plan_sections.items():
            if section_name == "Trajectory":  # Handle Trajectory with iterative prompting
                # Initial prompt
                prompt = section_instructions["initial_prompt"]
                trajectory_details = [generate_flight_plan_with_llm(prompt, system_prompt, self.model, temperature=0.7)]

                # Follow-up prompts
                for sub_question in section_instructions["follow_up_prompts"]:
                    prompt = f"""
                    Specifically, focus on the {section_name}:

                    {sub_question}
                    """
                    trajectory_details.append(generate_flight_plan_with_llm(prompt, system_prompt, self.model, temperature=0.7))

                flight_plan[section_name] = "\n".join(trajectory_details)  # Combine responses

            else:
                prompt = f"""
                Specifically, focus on the {section_name}:

                {section_instructions}
                """
                flight_plan[section_name] = generate_flight_plan_with_llm(prompt, system_prompt, self.model, temperature=0.7) # Added temperature parameter

        return {self.model: flight_plan} # Return single flight plan for current model



# Example usage
if __name__ == "__main__":
    agent = SpaceFlightPlanningAgent(llama_model_id)  # Using Llama model ID here

    all_flight_plans = agent.generate_flight_plan(
        origin="Earth",
        destination="Mars",
        spacecraft="Orion spacecraft",
        mission_objectives=["Land on Mars", "Collect soil samples"],
        constraints=["Minimize radiation exposure", "Ensure crew safety"]
    )

    # Print the flight plans generated by each LLM
    for model, flight_plan in all_flight_plans.items():
        print(f"Flight plan generated by {model}:")
        for section_name, section_content in flight_plan.items():
            print(f"{section_name}:\n{section_content}\n")

Flight plan generated by meta-llama/Llama-3.1-8B-Instruct:
Launch Date and Time:
Consider that Earth and Mars are in opposition every 26 months.
                 Provide a launch window between 2023-2028.
                 Provide a launch time that minimizes radiation exposure.
                 Use Hohmann transfer orbits and delta-v calculations to determine the optimal launch time.
                 Assume that the spacecraft can launch with a thrust-to-weight ratio of 0.3 and a specific impulse of 450 seconds.
                 Assume that the Earth-Mars opposition occurs on June 17, 2024.
                 Assume that the launch window is between 2-3 days prior to opposition, and that the launch time is within 12 hours of the opposition time.
                 Assume that the launch time with minimum radiation exposure is when the spacecraft is on the opposite side of the Sun from Earth.
                 Assume that the solar activity is moderate (Kp index = 5).

                Please

## EARTH TO MOON

In [3]:
import os
from typing import List, Dict
from datetime import datetime, timedelta
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer


def generate_flight_plan_with_llm(prompt: str, system_prompt: str = None, model: str = llama_model_id, temperature: float = 0.7, max_tokens=500) -> str:
    """Generates a response from the LLM."""
    full_prompt = prompt
    if system_prompt:
        full_prompt = f"{system_prompt}\n{prompt}"
    response = llama_pipeline(full_prompt, max_new_tokens=max_tokens, temperature=temperature, num_return_sequences=1)  # Using llama_pipeline
    flight_plan = response[0]['generated_text'].replace(full_prompt, "").strip()  # Extract generated content
    return flight_plan


def generate_flight_plan_iteratively(prompt: str, system_prompt: str = None, model: str = llama_model_id, temperature: float = 0.7, max_tokens_per_iteration=500) -> str:
    """Generates a flight plan iteratively, handling potential truncations with a response chunking mechanism."""

    streamer = TextStreamer(llama_tokenizer)  # For iterative generation
    full_response = ""
    current_prompt = prompt
    if system_prompt:
        current_prompt = f"{system_prompt}\n{prompt}"

    # Iterative generation
    for chunk in llama_pipeline(current_prompt, max_new_tokens=max_tokens_per_iteration, temperature=temperature, num_return_sequences=1, streamer=streamer):  # Using llama_pipeline
        full_response += chunk['generated_text']
    return full_response


# Define a class for the AI agent
class SpaceFlightPlanningAgent:
    def __init__(self, llm_models: List[str]):
        self.llm_models = llm_models

    def generate_flight_plan(
            self,
            origin: str,
            destination: str,
            spacecraft: str,
            mission_objectives: List[str],
            constraints: List[str],
    ) -> Dict[str, Dict[str, str]]:
        system_prompt = f"""
        You are a helpful and informative space flight planner.

        You are tasked with generating a detailed flight plan for a mission from {origin} to {destination}
        using a {spacecraft}. The mission objectives are: {mission_objectives}. The flight plan
        must satisfy the following constraints: {constraints}.

        Consider these factors:
        - Orbital mechanics (including translunar injection, lunar orbit insertion, and descent/ascent maneuvers)
        - Spacecraft capabilities (delta-v, thrust-to-weight ratio, specific impulse)
        - Gravitational forces of relevant celestial bodies (Earth, Moon)
        - Radiation exposure (minimize exposure by considering solar activity and trajectory)
        - Communication windows (ensure contact with Earth throughout the mission)
        - Launch opportunities (identify optimal launch windows based on lunar phases and Earth-Moon alignment)
        """

        flight_plan_sections = {
            "Launch Date and Time": f"""Provide a **specific** launch date and time in UTC, considering optimal launch windows and radiation exposure.
                                 Do not provide a range of dates or suggest future windows. Choose a date and time within the next 5 years,
                                 starting from {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}. """,  # Modified prompt to include current date and time
            # Trajectory broken down into subsections:
            "Trajectory": {
                "Launch": "Describe the launch process, including the launch vehicle, launch site, and initial parking orbit.",
                "Earth Departure and Translunar Injection": "Describe the Earth departure and translunar injection maneuvers, including the burn durations, delta-v requirements, and resulting trajectory.",
                "Lunar Orbit Insertion": "Describe the lunar orbit insertion maneuver, including the burn duration, delta-v requirements, and target lunar orbit.",
                "Lunar Descent and Landing": "Describe the lunar descent and landing process, including the landing site selection and landing procedures.",
                "Lunar Surface Operations": "Describe the planned activities on the lunar surface, including scientific experiments and sample collection.",
                "Lunar Ascent and Return": "Describe the lunar ascent and return trajectory, including the maneuvers and burn durations.",
                "Earth Re-entry and Landing": "Describe the Earth re-entry and landing process, including the atmospheric entry and splashdown or landing procedures."
            },
            "Maneuver Schedule": "Detail each maneuver with its purpose, delta-v requirement, and timing. Include maneuvers like translunar injection, lunar orbit insertion, descent, ascent, and any course correction burns.",
            "Communication Plan": "Explain how communication with Earth will be maintained throughout the mission, including the use of ground stations and any considerations for communication delays or blackouts.",
            "Contingency Plans": "Describe contingency plans for various scenarios, such as launch vehicle failure, missed maneuver opportunities, or spacecraft malfunctions. Address crew safety and mission recovery options."
        }

        all_flight_plans = {}  # Store flight plans for all models

        for model in self.llm_models:
            flight_plan = {}  # Store flight plan sections for current model
            for section_name, section_instructions in flight_plan_sections.items():

                # Handle Trajectory subsections:
                if section_name == "Trajectory":
                    trajectory_details = ""
                    for subsection_name, subsection_instructions in section_instructions.items():
                        prompt = f"""
                        Specifically, focus on the {subsection_name} of the Trajectory:

                        {subsection_instructions}
                        """
                        response = generate_flight_plan_iteratively(prompt, system_prompt, model, temperature=0.7)
                        trajectory_details += f"\n**{subsection_name}:**\n{response}\n"
                    flight_plan[section_name] = trajectory_details  # Store the combined trajectory details
                else:
                    # Your existing code for other sections remains the same
                    prompt = f"""
                    Specifically, focus on the {section_name}:

                    {section_instructions}
                    """
                    response = generate_flight_plan_with_llm(prompt, system_prompt, model, temperature=0.7)

                    # Response Evaluation and Adjustment (if needed):
                    retry_count = 0
                    temp = 0.7  # Initial temperature
                    while ("know" in response or "spend several months in this phase" in response or len(response) < 100) and retry_count < 3:
                        print(f"Response for {section_name} was truncated or incomplete. Retrying with temperature {temp}...")
                        # Retry with the same prompt, but use the original function to avoid infinite loop
                        response = generate_flight_plan_with_llm(prompt, system_prompt, model, temperature=temp)
                        retry_count += 1
                        temp += 0.1  # Increase temperature for next retry

                    flight_plan[section_name] = response  # Store the final (hopefully complete) response

                all_flight_plans[model] = flight_plan  # Add model's flight plan to the dictionary # Moved outside the inner loop

        return all_flight_plans

# Example usage
if __name__ == "__main__":
    agent = SpaceFlightPlanningAgent([llama_model_id])  # Using Llama model ID here

    all_flight_plans = agent.generate_flight_plan(
        origin="Earth",
        destination="Moon",
        spacecraft="Orion spacecraft",
        mission_objectives=["Land on the Moon", "Conduct scientific experiments"],
        constraints=["Minimize radiation exposure", "Ensure crew safety"]
    )

    # Print the flight plans generated by each LLM
    for model, flight_plan in all_flight_plans.items():
        print(f"Flight plan generated by {model}:")
        for section_name, section_content in flight_plan.items():
            print(f"{section_name}:\n{section_content}\n")

<|begin_of_text|>
        You are a helpful and informative space flight planner.

        You are tasked with generating a detailed flight plan for a mission from Earth to Moon
        using a Orion spacecraft. The mission objectives are: ['Land on the Moon', 'Conduct scientific experiments']. The flight plan
        must satisfy the following constraints: ['Minimize radiation exposure', 'Ensure crew safety'].

        Consider these factors:
        - Orbital mechanics (including translunar injection, lunar orbit insertion, and descent/ascent maneuvers)
        - Spacecraft capabilities (delta-v, thrust-to-weight ratio, specific impulse)
        - Gravitational forces of relevant celestial bodies (Earth, Moon)
        - Radiation exposure (minimize exposure by considering solar activity and trajectory)
        - Communication windows (ensure contact with Earth throughout the mission)
        - Launch opportunities (identify optimal launch windows based on lunar phases and Earth-Moon 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Flight plan generated by meta-llama/Llama-3.1-8B-Instruct:
Launch Date and Time:
# Step 1: Define the mission requirements and constraints
The mission objectives are to land on the Moon and conduct scientific experiments. The constraints are to minimize radiation exposure and ensure crew safety.

# Step 2: Determine the optimal launch date and time
To minimize radiation exposure, we should avoid periods of high solar activity. According to NASA's Space Weather Prediction Center, the solar activity is typically highest around solar maximum, which occurs approximately every 11 years.

# Step 3: Identify the optimal launch window
Given the constraints and mission objectives, we should identify a launch window that minimizes radiation exposure and ensures crew safety. Considering the lunar phases and Earth-Moon alignment, we should select a launch window that allows for a direct transfer to the Moon.

# Step 4: Choose a specific launch date and time
Based on the above considerations, let's